In [1]:
import requests
from fake_useragent import UserAgent
import re
import time
import random

In [2]:
def get_article_url_list (contry = 'venesuela', iter_number= 3):    
    date = '2024-09-23T15:05:01.994964'
    article_url_list =[]
    params = {        
        'user-agent': UserAgent().random       
    }
    for iter in range(1,iter_number):
        try:
            # Мали url = 'https://tass.ru/tbp/api/v1/content?limit=11&last_es_updated_dt={}&lang=ru&exclude_types=press_center&tags=/mali&sort=-es_updated_dt'.format(date)
            url = 'https://tass.ru/tbp/api/v1/content?limit=11&last_es_updated_dt={}&lang=ru&exclude_types=press_center&tags=/{}&sort=-es_updated_dt'.format(date, contry) 
         
            res = requests.get(url, data=params)
            #print(res.json())
            print('Извлечение url, {} итерация'.format(iter))
            for item in res.json()["result"]:
                #print('https://tass.ru'+item["url"])
                article_url_list.append('https://tass.ru'+item["url"])
                date = item["es_updated_dt"]
            time.sleep(random.randrange(3, 8))    
        except Exception as ex:
            print(ex)
        
    return article_url_list

In [3]:
import string

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content

import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
spec_chars = string.punctuation + '\r' + '\n\xa0«»\t—…' 


In [4]:
def get_text(r):
    # Извлечение текста из статьи
    #article_title - Название статьи
    #article_text - Текст статьи без ссылок
    #article_tags - метки статьи
   
    from bs4 import BeautifulSoup
    import re
    article_tags = []
    article_text = ''
    soup = BeautifulSoup(r.text, 'lxml')
    article_title = soup.find('h1', {'class': 'NewsHeader_titles__uKY5F'}).text
    article_body = soup.find_all('p', {'class': 'Paragraph_paragraph__9WAFK'})
    tags_item = soup.find_all('a', {'class': 'Tags_tag__o7Dqc'})
    #print (article_title)
    #print (article_body)
    #print (tags_item)
    for item in article_body:
        soup_temp = BeautifulSoup(item.text, 'lxml')
        text_post = re.sub(r'https?:\/\/.*[\r\n]*', '', soup_temp.text, flags=re.MULTILINE)
        text_post = remove_emojis(text_post)
        text_post = re.sub('\n\n', '\n', text_post, flags=re.MULTILINE)
        article_text = article_text + '\n'+ text_post    
    article_tags= [p.text for p in tags_item]    
    head_post, sep, article_text = article_text.partition('/ТАСС/.') # Удаление заголовка .../ТАСС/.
    return article_title, article_text, article_tags

In [5]:
def get_article(article_url, contry='venesuela'):
    import requests
    from fake_useragent import UserAgent
    import random
    import time
    from bs4 import BeautifulSoup
    url = article_url
    useragent = {'User-Agent': UserAgent().random}
    try:
        r = requests.get(url, headers=useragent)
        time.sleep(random.randrange(1, 6))
        article_title, article_text, article_tags = get_text(r)
        #print(article_title)       
        #print(article_text)
        #print(article_tags)
        # Запись в файл        
        try:
            with open('./Tass_Dataset_tag/{}/tass_{}.txt'.format( contry, re.search(r'(\d+)', url).group(1)), "w", encoding="utf-8") as some_file:
                print("\n\n".join([article_title, article_text]), file=some_file)
                some_file.close()
            with open('./Tass_Dataset_tag/tag/{}/tass_{}_tag.txt'.format( contry, re.search(r'(\d+)', url).group(1)), "w", encoding="utf-8") as some_file:
                print("\n\n".join(article_tags), file=some_file)
                some_file.close()                
        except Exception as ex:
            print(ex)
    except Exception as ex:
        print(ex)
    return 

In [6]:
#article_url_list = get_article_url_list(20)

In [7]:
#article_url_list[:10]

In [8]:
#len(article_url_list)

In [9]:
#for article_url in article_url_list:
#    get_article(article_url)

In [10]:
Afrika_tags = [ 'kitay', 'chad', 'gana', 'pakistan', 'vetnam', 'laos']

for contry in Afrika_tags:
    article_url_list = get_article_url_list(contry, 20)
    print(article_url_list[:10])
    for article_url in article_url_list:
        get_article(article_url, contry)
    time.sleep(random.randrange(10, 30))

Извлечение url, 1 итерация
Извлечение url, 2 итерация
Извлечение url, 3 итерация
Извлечение url, 4 итерация
Извлечение url, 5 итерация
Извлечение url, 6 итерация
Извлечение url, 7 итерация
Извлечение url, 8 итерация
Извлечение url, 9 итерация
Извлечение url, 10 итерация
Извлечение url, 11 итерация
Извлечение url, 12 итерация
Извлечение url, 13 итерация
Извлечение url, 14 итерация
Извлечение url, 15 итерация
Извлечение url, 16 итерация
Извлечение url, 17 итерация
Извлечение url, 18 итерация
Извлечение url, 19 итерация
['https://tass.ru/info/21934163', 'https://tass.ru/mezhdunarodnaya-panorama/21932323', 'https://tass.ru/mezhdunarodnaya-panorama/21931863', 'https://tass.ru/mezhdunarodnaya-panorama/21931265', 'https://tass.ru/obschestvo/21930991', 'https://tass.ru/ekonomika/21930941', 'https://tass.ru/obschestvo/21930779', 'https://tass.ru/mezhdunarodnaya-panorama/21930595', 'https://tass.ru/ekonomika/21930485', 'https://tass.ru/ekonomika/21929899']


C:\Users\user\AppData\Local\Temp\ipykernel_13004\2941976521.py:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup_temp = BeautifulSoup(item.text, 'lxml')


'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
Извлечение url, 1 итерация
Извлечение url, 2 итерация
Извлечение url, 3 итерация
Извлечение url, 4 итерация
Извлечение url, 5 итерация
Извлечение url, 6 итерация
Извлечение url, 7 итерация
Извлечение url, 8 итерация
Извлечение url, 9 итерация
Извлечение url, 10 итерация
Извлечение url, 11 итерация
Извлечение url, 12 итерация
Извлечение url, 13 итерация
Извлечение url, 14 итерация
Извлечение url, 15 итерация
Извлечение url, 16 итерация
Извлечение url, 17 итерация
Извлечение url, 18 итерация
Извлечение url, 19 итерация
['https://tass.ru/obschestvo/21927349', 'https://tass.ru/proisshestviya/21924543', 'https://tass.ru/obschestvo/21874219', 'https://tass.ru/mezhdunarodnaya-panorama/21865971', 'https://tass.ru/proisshestviya/21825431', 'https://tass.ru/mezhdunarodnaya-panorama/2179645